In [26]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup
from jim_decoding_functions import *


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [27]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# subjects = ['D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [28]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [29]:
task='GlobalLocal'

conditions = stimulus_experiment_conditions # toggle
# conditions = stimulus_conditions # toggle

if conditions == stimulus_conditions or stimulus_experiment_conditions:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
elif conditions == response_conditions or response_experiment_conditions:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
else:
    raise ValueError("Unknown condition type.")

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_HG_ev1_power_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus/i25.0/s25.0 with parameters: {'BIDS_events': 'Stimulus/i25.0/s25.0',

In [30]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/i25.0/s25.0',
 'Stimulus/i25.0/s75.0',
 'Stimulus/i75.0/s25.0',
 'Stimulus/i75.0/s75.0',
 'Stimulus/i25.0/r25.0',
 'Stimulus/i25.0/r75.0',
 'Stimulus/i75.0/r25.0',
 'Stimulus/i75.0/r75.0',
 'Stimulus/c25.0/s25.0',
 'Stimulus/c25.0/s75.0',
 'Stimulus/c75.0/s25.0',
 'Stimulus/c75.0/s75.0',
 'Stimulus/c25.0/r25.0',
 'Stimulus/c25.0/r75.0',
 'Stimulus/c75.0/r25.0',
 'Stimulus/c75.0/r75.0']

get significant channels

In [31]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [32]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF

let's manually make the occ filtered sig electrodes

In [33]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [34]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [35]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [36]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 164
Total number of significant lpfc electrodes across all subjects: 59
Total number of v1 electrodes across all subjects: 19
Total number of significant v1 electrodes across all subjects: 11
Total number of occ electrodes across all subjects: 76
Total number of significant occ electrodes across all subjects: 51
Total number of occ_filtered electrodes across all subjects: 0
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 0
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [37]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [38]:
# Example call to the function (you need to replace the arguments with actual data)
# dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.


some helper functions

In [39]:
def get_max_trials_per_condition(
    subjects_mne_objects, condition_names, subjects,
    sig_electrodes_per_subject_roi, roi, obs_axs
):
    """
    Find the maximum number of trials per condition across all subjects for a given ROI,
    and identify which subject(s) have that maximum number of trials.
    """
    max_trials_per_condition = {condition: 0 for condition in condition_names}
    max_trials_subject_per_condition = {condition: [] for condition in condition_names}

    for sub in subjects:
        sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        if not sig_electrodes:
            continue
        for condition_name in condition_names:
            # Check if the subject has data for this condition
            if condition_name not in subjects_mne_objects[sub]:
                continue
            epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_power_rescaled'].copy().pick(sig_electrodes)
            epochs_data = epochs.get_data(copy=True)
            n_trials = epochs_data.shape[obs_axs]
            if n_trials > max_trials_per_condition[condition_name]:
                max_trials_per_condition[condition_name] = n_trials
                max_trials_subject_per_condition[condition_name] = [sub]
            elif n_trials == max_trials_per_condition[condition_name]:
                max_trials_subject_per_condition[condition_name].append(sub)
    return max_trials_per_condition, max_trials_subject_per_condition

def make_subject_labeled_array(
    sub, subjects_mne_objects, condition_names, sig_electrodes_per_subject_roi,
    roi, max_trials_per_condition, obs_axs, chans_axs, time_axs, rng
):
    """
    Process data for a subject in a given ROI.

    This function performs the following steps for the given subject:
    1. Retrieves the significant electrodes in the specified ROI for the subject.
    2. For each condition:
        a. Extracts the epoch data for the significant electrodes.
        b. Randomizes the trial order using the provided random state.
        c. Pads the data with NaNs to match the maximum number of trials for that condition.
    3. Collects the processed data for all conditions into a dictionary.
    4. Creates a LabeledArray from the processed data, assigning appropriate channel and time labels.

    Returns a LabeledArray for the subject.
    """
    sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
    if not sig_electrodes:
        return None

    subject_nested_dict = {}

    # Get channel names for this subject's ROI
    sub_channel_names = [f"{sub}-{sig_electrode}" for sig_electrode in sig_electrodes]

    # Loop through each condition
    for condition_name in condition_names:
        # Extract the epoch data for the current condition and subject
        epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_power_rescaled'].copy().pick(sig_electrodes)
        epochs_data = epochs.get_data(copy=True)

        # Randomize the trial order
        n_trials = epochs_data.shape[obs_axs]
        print(f'in roi {roi}, subject {sub} has {n_trials} trials for condition {condition_name}')
        trial_indices = np.arange(n_trials)
        rng.shuffle(trial_indices)
        epochs_data = epochs_data.take(trial_indices, axis=obs_axs)

        # Get the target number of trials for padding
        max_trials = max_trials_per_condition[condition_name]

        # Pad with NaNs if necessary
        if n_trials < max_trials:
            padded_shape = list(epochs_data.shape)
            padded_shape[obs_axs] = max_trials
            padded_data = np.full(padded_shape, np.nan)
            indexer = [slice(None)] * epochs_data.ndim
            indexer[obs_axs] = slice(0, n_trials)
            padded_data[tuple(indexer)] = epochs_data
        else:
            padded_data = epochs_data

        subject_nested_dict[condition_name] = padded_data

    # Get time labels
    times = epochs.times
    str_times = [str(time) for time in times]
    np_array_str_times = np.array(str_times)

    # Create a LabeledArray for the subject
    subject_labeled_array = create_subject_labeled_array_from_dict(
        subject_nested_dict, sub_channel_names, np_array_str_times, chans_axs, time_axs
    )

    # Print the shape and time axis labels
    print(f"Subject {sub}, ROI {roi}, LabeledArray shape: {subject_labeled_array.shape}")
    time_axis_size = subject_labeled_array.shape[time_axs+1] # Adjusted time axis index

    return subject_labeled_array

def create_subject_labeled_array_from_dict(
    subject_nested_dict, sub_channel_names, np_array_str_times, chans_axs, time_axs
):
    """
    Create a LabeledArray for a subject with given data and labels.
    This accounts for the conditions axis by incrementing the channels and time axes by 1.
    """
    subject_labeled_array = LabeledArray.from_dict(subject_nested_dict)
    # Adjust axes indices due to the added conditions axis
    subject_labeled_array.labels[chans_axs + 1].values = sub_channel_names  # Channels axis
    subject_labeled_array.labels[time_axs + 1].values = np_array_str_times  # Time axis
    return subject_labeled_array

def concatenate_subject_labeled_arrays(
    roi_labeled_array, subject_labeled_array, chans_axs
):
    """
    Concatenate a subject's LabeledArray to the ROI's LabeledArray along the channels axis.
    Adjusts for the conditions axis by incrementing chans_axs by 1.
    """
    concatenation_axis = chans_axs + 1  # Adjusted channels axis index
    if roi_labeled_array is None:
        return subject_labeled_array
    else:
        return roi_labeled_array.concatenate(subject_labeled_array, axis=concatenation_axis)

main function

In [40]:
def put_data_in_labeled_array_per_roi_subject(
    subjects_mne_objects, condition_names, rois, subjects,
    sig_electrodes_per_subject_roi, obs_axs=0, chans_axs=1, time_axs=2,
    random_state=None
):
    """
    Organize the MNE data into separate LabeledArrays for each ROI and subject,
    with randomized trial ordering within each subject before concatenation.
    Concatenates subject data along the channels axis.

    Parameters:
    - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}
    - condition_names: List of condition names.
    - rois: List of region of interest (ROI) names.
    - subjects: List of subjects.
    - sig_electrodes_per_subject_roi: Dictionary mapping ROIs to subjects and their corresponding electrodes.
    - obs_axs: The trials dimension (ignoring the conditions dimension for now)
    - chans_axs: The channels dimension
    - time_axs: The time dimension
    - random_state: Optional; an integer seed, NumPy RandomState, or None for random shuffling.

    Returns:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
                          Each LabeledArray has dimensions: [Conditions, Trials, Channels, Timepoints]
    """
    # Set up the random state
    rng = np.random.RandomState(random_state)
    roi_labeled_arrays = {}

    # Loop through each ROI
    for roi in rois:
        # First pass: Find the max number of trials per condition across all subjects
        max_trials_per_condition, max_trials_subject_per_condition = get_max_trials_per_condition(
            subjects_mne_objects, condition_names, subjects,
            sig_electrodes_per_subject_roi, roi, obs_axs
        )

        # Print out the subjects with maximum trials
        print(f"ROI '{roi}': Maximum trials per condition:")
        for condition_name in condition_names:
            max_trials = max_trials_per_condition[condition_name]
            subjects_with_max_trials = max_trials_subject_per_condition[condition_name]
            print(f"  Condition '{condition_name}': {max_trials} trials from subjects {subjects_with_max_trials}")
            
        # Initialize the ROI LabeledArray
        roi_labeled_array = None

        # Second pass: Process each subject's data
        for sub in subjects:
            subject_labeled_array = make_subject_labeled_array(
                sub, subjects_mne_objects, condition_names, sig_electrodes_per_subject_roi,
                roi, max_trials_per_condition, obs_axs, chans_axs, time_axs, rng
            )
            if subject_labeled_array is None:
                continue  # Skip if subject has no data for this ROI

            # Concatenate subject's data into the ROI LabeledArray
            roi_labeled_array = concatenate_subject_labeled_arrays(
                roi_labeled_array, subject_labeled_array, chans_axs
            )

        # Add the concatenated LabeledArray to the ROI dictionary
        if roi_labeled_array is not None:
            roi_labeled_arrays[roi] = roi_labeled_array

    return roi_labeled_arrays

In [41]:
# Example usage:
roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(
    subjects_mne_objects,
    condition_names,
    rois,
    subjects,
    sig_electrodes_per_subject_roi,
    obs_axs=0,  # Trials dimension (ignoring the conditions dimension for now cuz we iterate over it)
    chans_axs=1,  # Channels dimension
    time_axs=2,   # Time dimension
    random_state=42  # For reproducibility
)

ROI 'lpfc': Maximum trials per condition:
  Condition 'Stimulus/i25.0/s25.0': 7 trials from subjects ['D0057']
  Condition 'Stimulus/i25.0/s75.0': 21 trials from subjects ['D0059', 'D0090']
  Condition 'Stimulus/i75.0/s25.0': 22 trials from subjects ['D0057']
  Condition 'Stimulus/i75.0/s75.0': 63 trials from subjects ['D0057']
  Condition 'Stimulus/i25.0/r25.0': 22 trials from subjects ['D0071', 'D0090', 'D0103']
  Condition 'Stimulus/i25.0/r75.0': 7 trials from subjects ['D0090']
  Condition 'Stimulus/i75.0/r25.0': 62 trials from subjects ['D0059']
  Condition 'Stimulus/i75.0/r75.0': 23 trials from subjects ['D0071']
  Condition 'Stimulus/c25.0/s25.0': 23 trials from subjects ['D0103']
  Condition 'Stimulus/c25.0/s75.0': 60 trials from subjects ['D0063', 'D0094']
  Condition 'Stimulus/c75.0/s25.0': 9 trials from subjects ['D0059']
  Condition 'Stimulus/c75.0/s75.0': 22 trials from subjects ['D0059', 'D0071']
  Condition 'Stimulus/c25.0/r25.0': 64 trials from subjects ['D0063']
  Cond

In [42]:
roi_labeled_arrays['lpfc'].shape

(16, 64, 59, 641)

remove trials with nans from roi labeled arrays if just getting the minimum number of trials for all subjects

In [43]:
def remove_nans_from_labeled_array(labeled_array, obs_axs=0, chans_axs=1, time_axs=2):
    """
    Remove trials that have NaN values from a LabeledArray and identify conditions with no valid trials.

    Parameters:
    - labeled_array: A LabeledArray with conditions, trials, channels, and timepoints dimensions.
    - obs_axs: The trials dimension 
    - chans_axs: The channels dimension
    - time_axs: The time dimension

    Returns:
    - labeled_array_no_nans: A LabeledArray with only trials that have no NaN values.
    - conditions_with_no_valid_trials: List of condition names with no valid trials after NaN removal.
    """

    # Initialize a dictionary to store data without NaNs for each condition
    reshaped_data_dict = {}
    # Initialize a list to keep track of conditions with no valid trials
    conditions_with_no_valid_trials = []

    # Extract the condition labels (which are in labeled_array.labels[0])
    condition_names = labeled_array.labels[0]  # Ensure we get the condition names

    # Loop over each condition
    for condition_name in condition_names:
        # Extract the data for the current condition
        condition_data = labeled_array[condition_name]  # Shape: (Trials, Channels, Timepoints)

        # Find the indices of trials that do not contain NaNs
        # Reduce over channels and time axes to check if any NaN exists in a trial
        valid_trial_indices = ~np.isnan(condition_data).any(axis=(chans_axs, time_axs))

        # valid_trial_indices is a boolean array of shape (Trials,)
        # Select only the valid trials
        condition_data_clean = condition_data[valid_trial_indices, :, :]

        # Check if there are valid trials
        if condition_data_clean.shape[obs_axs] > 0:
            # Store the processed data for this condition
            reshaped_data_dict[condition_name] = condition_data_clean
        else:
            print(f"No valid trials for condition '{condition_name}' after removing NaNs.")
            conditions_with_no_valid_trials.append(condition_name)

    # Optionally, print the conditions with no valid trials
    if conditions_with_no_valid_trials:
        print("Conditions with no valid trials after NaN removal:", conditions_with_no_valid_trials)

    # Proceed to create the LabeledArray if there are any valid conditions
    if len(reshaped_data_dict) == 0:
        raise ValueError("All conditions have no valid trials after removing NaNs.")

    # Create a new LabeledArray from the reshaped data dictionary
    labeled_array_no_nans = LabeledArray.from_dict(reshaped_data_dict)

    return labeled_array_no_nans, conditions_with_no_valid_trials


def remove_nans_from_all_roi_labeled_arrays(roi_labeled_arrays, obs_axs=0, chans_axs=1, time_axs=2):
    """
    Loop through all ROIs and apply the NaN removal function to each LabeledArray.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - obs_axs: The trials dimension.
    - chans_axs: The channels dimension.
    - time_axs: The time dimension.

    Returns:
    - roi_labeled_arrays_no_nans: Dictionary where keys are ROIs and values are LabeledArrays with NaNs removed.
    - conditions_with_no_valid_trials_per_roi: Dictionary where keys are ROIs and values are lists of condition names with no valid trials.
    """
    roi_labeled_arrays_no_nans = {}
    conditions_with_no_valid_trials_per_roi = {}

    # Loop through each ROI in the dictionary
    for roi, labeled_array in roi_labeled_arrays.items():
        # Apply the NaN removal function to the current labeled array
        labeled_array_no_nans, conditions_with_no_valid_trials = remove_nans_from_labeled_array(
            labeled_array, obs_axs=obs_axs, chans_axs=chans_axs, time_axs=time_axs
        )

        # Store the reshaped data for this ROI
        roi_labeled_arrays_no_nans[roi] = labeled_array_no_nans

        # Store the conditions with no valid trials for this ROI
        if conditions_with_no_valid_trials:
            conditions_with_no_valid_trials_per_roi[roi] = conditions_with_no_valid_trials

    return roi_labeled_arrays_no_nans, conditions_with_no_valid_trials_per_roi

In [44]:
roi_labeled_arrays_no_nans, conditions_with_no_valid_trials_per_roi = remove_nans_from_all_roi_labeled_arrays(roi_labeled_arrays, obs_axs=0, chans_axs=1, time_axs=2)

No valid trials for condition 'Stimulus/i25.0/s25.0' after removing NaNs.
No valid trials for condition 'Stimulus/i25.0/r75.0' after removing NaNs.
Conditions with no valid trials after NaN removal: ['Stimulus/i25.0/s25.0', 'Stimulus/i25.0/r75.0']


In [45]:
'''
11/6 need to update the obs_axs here to work with obs_axs=1 and also all the way down too 
(though I'm not sure why the obs_axs is 0 for decoding, maybe cuz I make arrays for each condition?)
'''
def concatenate_conditions_by_string(roi_labeled_arrays, roi, strings_to_find, obs_axs=0):
    """
    Concatenate trials across condition names that contain specific strings.
    Assign labels based on the groupings of the conditions.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - roi: The specific ROI to process.
    - strings_to_find: List of strings or list of lists of strings to search for in condition names.
                      If a list of strings is provided, each string is treated as its own condition group.
                      If a list of lists is provided, each sublist represents a group of conditions.
    - obs_axs (int) : The trials dimension. Concatenation will happen along this axis. This is the 1st dimension (not 0th) because conditions in the labeled array is the 0th. But we will subtract 1 if not considering conditions as a dimension (looping over conditions)
    
    Returns:
    - concatenated_data: The concatenated trials by (channels, timepoints, or whatever your other dimensions are) across the matching conditions.
    - labels: A numpy array of labels (0, 1, 2, ...) corresponding to each group of conditions.
    - cats: Dictionary of {condition_name: index} for decoding.
    """
    concatenated_data = []
    labels = []
    cats = {}

    # Track current label index
    current_label = 0

    # Normalize strings_to_find so each entry is a list (whether it's a string or a list of strings)
    if isinstance(strings_to_find, list) and all(isinstance(s, str) for s in strings_to_find):
        # If it's a flat list of strings, convert each string into its own single-item list
        strings_to_find = [[s] for s in strings_to_find]

    # Iterate over each group (whether it's a single string or a list of strings)
    for string_group in strings_to_find:
        # Find condition names that match any of the strings in the current string_group
        matching_conditions = [cond for cond in roi_labeled_arrays[roi].keys() if any(s in cond for s in string_group)]

        if not matching_conditions:
            continue

        # Concatenate data for all matching conditions
        data_to_concatenate = []
        for cond in matching_conditions:
            # Extract data for the current condition
            data = roi_labeled_arrays[roi][cond]  # Shape: (trials, channels, timepoints)
            data_to_concatenate.append(data)
            
            # Update labels for the current condition group
            labels.extend([current_label] * data.shape[0])
        
        # Check if we have data to concatenate for this condition group
        if data_to_concatenate:
            concatenated_data.append(np.concatenate(data_to_concatenate, axis=obs_axs))

        # Assign current label to the condition group (based on the first string in the group for reference)
        cats[tuple(string_group)] = current_label
        current_label += 1

    # Ensure there is data to concatenate
    if not concatenated_data:
        raise ValueError(f"No matching conditions found for ROI: {roi} and strings: {strings_to_find}")

    # Concatenate all condition data along the trials axis
    concatenated_data = np.concatenate(concatenated_data, axis=obs_axs)
    
    return concatenated_data, np.array(labels), cats

set up decoding output directory and conditions to compare 9/12

In [46]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'decoding', 'figs')

if conditions == stimulus_experiment_conditions:
    condition_comparisons = {}
    condition_comparisons['congruency'] = [['c25.0', 'c75.0'], ['i25.0', 'i75.0']]
    condition_comparisons['switchType'] = [['r25.0', 'r75.0'], ['s25.0', 's75.0']]
    
elif conditions == stimulus_conditions:
    condition_comparisons = {}
    condition_comparisons['BigLetter'] = ['BigLetters', 'BigLetterh']
    condition_comparisons['SmallLetter'] = ['SmallLetters', 'SmallLetterh']
    condition_comparisons['Task'] = ['Taskg', 'Taskl']
    
    # strings_to_find = ['BigLetters', 'BigLetterh']
    # condition_0_name = 'BigLetters'
    # condition_1_name = 'BigLetterh'

    # strings_to_find = ['SmallLetters', 'SmallLetterh']
    # condition_0_name = 'SmallLetters'
    # condition_1_name = 'SmallLetterh'

    # strings_to_find = ['Taskg', 'Taskl']
    # condition_0_name = 'Taskg'
    # condition_1_name = 'Taskl'

do decoding on the reshaped time indexed arrays 9/12

let's try a class for storing and plotting the decoding scores for each roi 9/13

In [47]:
def get_and_plot_confusion_matrix_for_rois_jim(
    roi_labeled_arrays, rois, condition_comparison, strings_to_find, save_dir,
    time_interval_name=None, n_splits=5, n_repeats=5, obs_axs=0, balance_method='pad_with_nans', random_state=42
):
    """
    Compute the confusion matrix for each ROI and return it. This function allows for balancing trial counts
    either by padding with NaNs or by subsampling trials to match the condition with the fewest valid (non-NaN) trials.
    
    Parameters:
    - roi_labeled_arrays: Dictionary containing the reshaped data for each ROI.
    - rois: List of regions of interest (ROIs) to process.
    - condition_comparison: The condition that we're comparing labels for (e.g., 'BigLetter').
    - strings_to_find: List of strings or string groups to identify condition labels.
    - save_dir: Directory to save the confusion matrix plots.
    - time_interval_name: Optional string to add to the filename for the time interval.
    - n_splits: Number of splits for cross-validation.
    - n_repeats: Number of repeats for cross-validation.
    - obs_axs: The trials axis.
    - balance_method: 'pad_with_nans' or 'subsample' to balance trial counts between conditions.
    - random_state: Random seed for reproducibility.
    
    Returns:
    - confusion_matrices: Dictionary containing confusion matrices for each ROI.
    """
    confusion_matrices = {}
    rng = np.random.RandomState(random_state)

    for roi in rois:
        print(f"Processing ROI: {roi}")
        
        # Concatenate the trials and get labels
        concatenated_data, labels, cats = concatenate_conditions_by_string(
            roi_labeled_arrays, roi, strings_to_find, obs_axs
        )

        print(f"Concatenated data shape for {roi}: {concatenated_data.shape}")

        if balance_method == 'subsample':
            # Remove NaN trials from concatenated_data and labels
            # Identify trials that contain NaNs
            nan_trials = np.isnan(concatenated_data).any(axis=tuple(range(1, concatenated_data.ndim)))
            valid_trials = ~nan_trials

            # Keep only valid trials
            concatenated_data = concatenated_data[valid_trials]
            labels = labels[valid_trials]

        # Calculate trial counts per condition
        trial_counts = {}
        condition_indices = {}

        for string_group in strings_to_find:
            # Get the label associated with the current string group
            condition_label = cats[tuple(string_group) if isinstance(string_group, list) else (string_group,)]
            
            # Select trials for the current condition group
            condition_trials = labels == condition_label
            data_for_condition = concatenated_data[condition_trials]
            
            # Store indices of trials for the current condition
            condition_indices[condition_label] = np.where(condition_trials)[0]

            # Count total trials for the current condition
            trial_counts[condition_label] = data_for_condition.shape[0]
            
            print(f'Condition {string_group} has {trial_counts[condition_label]} trials')

        if balance_method == 'pad_with_nans':
            # Determine the maximum trial count across conditions
            max_trial_count = max(trial_counts.values())
            
            # Balance trial counts by adding NaN trials where needed
            for condition_label, count in trial_counts.items():
                trials_to_add = max_trial_count - count
                if trials_to_add > 0:
                    nan_trial_shape = (trials_to_add,) + concatenated_data.shape[1:]
                    nan_trials = np.full(nan_trial_shape, np.nan)
                    concatenated_data = np.concatenate([concatenated_data, nan_trials], axis=obs_axs)
                    labels = np.concatenate([labels, [condition_label] * trials_to_add])
        elif balance_method == 'subsample':
            # Determine the minimum trial count across conditions
            min_trial_count = min(trial_counts.values())

            # Subsample trials from each condition to have min_trial_count trials
            subsampled_indices = []
            for condition_label in trial_counts.keys():
                indices = condition_indices[condition_label]
                if trial_counts[condition_label] > min_trial_count:
                    # Randomly select min_trial_count trials
                    selected_indices = rng.choice(indices, size=min_trial_count, replace=False)
                else:
                    selected_indices = indices
                subsampled_indices.extend(selected_indices)
            
            # Select the subsampled data and labels
            subsampled_indices = np.array(subsampled_indices)
            concatenated_data = concatenated_data[subsampled_indices]
            labels = labels[subsampled_indices]
        else:
            raise ValueError(f"Invalid balance_method: {balance_method}. Choose 'pad_with_nans' or 'subsample'.")

        # Create a Decoder and run cross-validation
        decoder = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_repeats)

        # Use the concatenated data for the decoder
        cm = decoder.cv_cm_jim(concatenated_data, labels, normalize='true', obs_axs=obs_axs)
        cm_avg = np.mean(cm, axis=0)

        # Store the confusion matrix in the dictionary
        confusion_matrices[roi] = cm_avg

        # Convert tuple labels to simple strings for display
        display_labels = [
            key[0] if isinstance(key, tuple) and len(key) == 1 else str(key)
        for key in cats.keys()
        ]
        
        # Plot the Confusion Matrix
        fig, ax = plt.subplots()
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_avg, display_labels=display_labels)
        disp.plot(ax=ax)

        # Save the figure with the time interval in the filename
        time_str = f"_{time_interval_name}" if time_interval_name else ""
        file_name = (
            f'{roi}_{condition_comparison}{time_str}_time_averaged_confusion_matrix_'
            f'{n_splits}splits_{n_repeats}repeats_{balance_method}.png'
        )
        # plt.savefig(os.path.join(save_dir, file_name))
        plt.close()

    return confusion_matrices


In [23]:
# get the confusion matrix using the downsampled version
for condition_comparison, strings_to_find in condition_comparisons.items():
    confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
        roi_labeled_arrays=roi_labeled_arrays_no_nans,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        n_splits=5,
        n_repeats=10,
        obs_axs=0,
        balance_method='subsample',  # Use 'subsample' to balance by subsampling
        random_state=42  # For reproducibility
    )

Processing ROI: lpfc
Concatenated data shape for lpfc: (658, 59, 641)
Condition ['c25.0', 'c75.0'] has 131 trials
Condition ['i25.0', 'i75.0'] has 72 trials
Processing ROI: v1
Concatenated data shape for v1: (864, 11, 641)
Condition ['c25.0', 'c75.0'] has 159 trials
Condition ['i25.0', 'i75.0'] has 157 trials
Processing ROI: occ
Concatenated data shape for occ: (608, 51, 641)
Condition ['c25.0', 'c75.0'] has 120 trials
Condition ['i25.0', 'i75.0'] has 84 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (736, 23, 641)
Condition ['c25.0', 'c75.0'] has 148 trials
Condition ['i25.0', 'i75.0'] has 95 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (864, 10, 641)
Condition ['c25.0', 'c75.0'] has 177 trials
Condition ['i25.0', 'i75.0'] has 103 trials
Processing ROI: lpfc
Concatenated data shape for lpfc: (658, 59, 641)
Condition ['r25.0', 'r75.0'] has 111 trials
Condition ['s25.0', 's75.0'] has 92 trials
Processing ROI: v1
C

In [24]:
# get the confusion matrix using the version with nan trials and doing nan trial mixup
for condition_comparison, strings_to_find in condition_comparisons.items():
    confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
        roi_labeled_arrays=roi_labeled_arrays,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        n_splits=5,
        n_repeats=10,
        obs_axs=0,
        balance_method='pad_with_nans',  # Use 'pad_with_nans' to balance by padding with nans
        random_state=42  # For reproducibility
    )


Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 59, 641)
Condition ['c25.0', 'c75.0'] has 512 trials
Condition ['i25.0', 'i75.0'] has 512 trials
Processing ROI: v1
Concatenated data shape for v1: (1008, 11, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ
Concatenated data shape for occ: (1008, 51, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (1008, 23, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (1008, 10, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 59, 641)
Condition ['r25.0', 'r75.0'] has 512 trials
Condition ['s25.0', 's75.0'] has 512 trials
Processing

prestimulus vs poststimulus confusion matrices 9/21

In [68]:
def get_data_in_time_range(labeled_array, time_range):
    """
    Extract data from a LabeledArray where the time points fall within a given range, using the LabeledArray `take` function.
    
    Parameters:
    - labeled_array: The LabeledArray containing time points as labels.
    - time_range: A tuple (start_time, end_time) representing the range of time.
    
    Returns:
    - filtered_data: A LabeledArray containing only the data where the timepoints are within the specified range.
    """
    start_time, end_time = time_range

    # Assume that the time labels are stored in the 4th dimension (index 3)
    time_points = np.array(labeled_array.labels[3], dtype=float)  # convert to floats, ensure conversion to float numpy array

    # Find the indices of time points within the specified range
    time_indices = np.where((time_points >= start_time) & (time_points <= end_time))[0]

    # Use the take function to select the time indices along the time axis (axis=3)
    filtered_data = labeled_array.take(time_indices, axis=3)

    return filtered_data


In [31]:
# Define the time intervals of interest
pre_stimulus_interval = (-1.0, 0.0)  # Pre-stimulus: time -1 to 0 seconds
post_stimulus_interval = (0.0, 1.5)  # Post-stimulus: time 0 to 1.5 seconds
pre_stimulus_roi_labeled_arrays = {}
post_stimulus_roi_labeled_arrays = {}

for roi, labeled_array in roi_labeled_arrays.items():
    pre_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, pre_stimulus_interval)
    post_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, post_stimulus_interval)


pre_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
    roi_labeled_arrays=pre_stimulus_roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name='pre_stimulus',  # If not dealing with specific time intervals
    n_splits=5,
    n_repeats=10
)

post_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
    roi_labeled_arrays=post_stimulus_roi_labeled_arrays,
    rois=rois,
    strings_to_find=strings_to_find,
    condition_0_name=condition_0_name,
    condition_1_name=condition_1_name,
    save_dir=save_dir,
    time_interval_name='post_stimulus',  # If not dealing with specific time intervals
    n_splits=5,
    n_repeats=10
)

Processing ROI: lpfc
Concatenated data shape for lpfc: (512, 59, 257)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: v1
Concatenated data shape for v1: (472, 11, 257)
Taskg has 236 trials
Taskl has 236 trials
Processing ROI: occ
Concatenated data shape for occ: (512, 51, 257)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (512, 23, 257)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (472, 10, 257)
Taskg has 236 trials
Taskl has 236 trials
Processing ROI: lpfc
Concatenated data shape for lpfc: (512, 59, 385)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: v1
Concatenated data shape for v1: (472, 11, 385)
Taskg has 236 trials
Taskl has 236 trials
Processing ROI: occ
Concatenated data shape for occ: (512, 51, 385)
Taskg has 256 trials
Taskl has 256 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (51

define colors for plotting (not used yet as of 8/21)

In [32]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}

new attempt at time window sliding decoding with significance 11/23  
TODO: Clean this up.   
Make subfunctions to break this down. Everything before defining the Decoder objects can be a function, that can be shared between this and the whole time window version.   
The decoder true and decoder shuffle can be done with a function maybe.   
And maybe return just accuracies, which I can then call this entire function separately for true and shuffled.  

  
ALSO STORE THE SHUFFLED OUTPUT IN A NUMPY ARRAY SO I DON'T HAVE TO MAKE IT EVERY TIME

In [48]:
def get_accuracies_for_rois_time_window_decoding_jim(
    roi_labeled_arrays, rois, condition_comparison, strings_to_find, save_dir,
    time_interval_name=None, n_splits=5, n_repeats=5, obs_axs=0, balance_method='pad_with_nans',
    random_state=42, window_size=None, step_size=1, n_permutations=100
):
    # Initialize dictionaries to store accuracies for each ROI
    accuracies_true_per_roi = {}
    accuracies_shuffle_per_roi = {}
    rng = np.random.RandomState(random_state)

    for roi in rois:
        print(f"Processing ROI: {roi}")
        
        # Concatenate the trials and get labels
        concatenated_data, labels, cats = concatenate_conditions_by_string(
            roi_labeled_arrays, roi, strings_to_find, obs_axs
        )

        print(f"Concatenated data shape for {roi}: {concatenated_data.shape}")

        if balance_method == 'subsample':
            # Remove NaN trials from concatenated_data and labels
            # Identify trials that contain NaNs
            nan_trials = np.isnan(concatenated_data).any(axis=tuple(range(1, concatenated_data.ndim)))
            valid_trials = ~nan_trials

            # Keep only valid trials
            concatenated_data = concatenated_data[valid_trials]
            labels = labels[valid_trials]

        # Calculate trial counts per condition
        trial_counts = {}
        condition_indices = {}

        for string_group in strings_to_find:
            # Get the label associated with the current string group
            condition_label = cats[tuple(string_group) if isinstance(string_group, list) else (string_group,)]
            
            # Select trials for the current condition group
            condition_trials = labels == condition_label
            data_for_condition = concatenated_data[condition_trials]
            
            # Store indices of trials for the current condition
            condition_indices[condition_label] = np.where(condition_trials)[0]

            # Count total trials for the current condition
            trial_counts[condition_label] = data_for_condition.shape[0]
            
            print(f'Condition {string_group} has {trial_counts[condition_label]} trials')

        if balance_method == 'pad_with_nans':
            # Determine the maximum trial count across conditions
            max_trial_count = max(trial_counts.values())
            
            # Balance trial counts by adding NaN trials where needed
            for condition_label, count in trial_counts.items():
                trials_to_add = max_trial_count - count
                if trials_to_add > 0:
                    nan_trial_shape = (trials_to_add,) + concatenated_data.shape[1:]
                    nan_trials = np.full(nan_trial_shape, np.nan)
                    concatenated_data = np.concatenate([concatenated_data, nan_trials], axis=obs_axs)
                    labels = np.concatenate([labels, [condition_label] * trials_to_add])
        elif balance_method == 'subsample':
            # Determine the minimum trial count across conditions
            min_trial_count = min(trial_counts.values())

            # Subsample trials from each condition to have min_trial_count trials
            subsampled_indices = []
            for condition_label in trial_counts.keys():
                indices = condition_indices[condition_label]
                if trial_counts[condition_label] > min_trial_count:
                    # Randomly select min_trial_count trials
                    selected_indices = rng.choice(indices, size=min_trial_count, replace=False)
                else:
                    selected_indices = indices
                subsampled_indices.extend(selected_indices)
            
            # Select the subsampled data and labels
            subsampled_indices = np.array(subsampled_indices)
            concatenated_data = concatenated_data[subsampled_indices]
            labels = labels[subsampled_indices]
        else:
            raise ValueError(f"Invalid balance_method: {balance_method}. Choose 'pad_with_nans' or 'subsample'.")
        
        # Create a Decoder instance
        decoder_true = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_repeats)
        decoder_shuffle = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_permutations)

        # Run decoding with true labels
        cm_true = decoder_true.cv_cm_jim_window_shuffle(
            concatenated_data, labels, normalize='true', obs_axs=obs_axs,
            window=window_size, step_size=step_size, shuffle=False
        )

        # Compute accuracies for each repetition
        accuracies_true = np.array([
            [np.trace(cm_true[win_idx, rep_idx]) / np.sum(cm_true[win_idx, rep_idx])
             for rep_idx in range(decoder_true.n_repeats)]
            for win_idx in range(cm_true.shape[0])
        ])  # Shape: (n_windows, n_repeats)

        # Mean accuracies across repetitions
        mean_accuracies_true = np.mean(accuracies_true, axis=1)  # Shape: (n_windows,)

        # Run decoding with shuffled labels
        cm_shuffle = decoder_shuffle.cv_cm_jim_window_shuffle(
            concatenated_data, labels, normalize='true', obs_axs=obs_axs,
            window=window_size, step_size=step_sze, shuffle=True
        )

        # Compute accuracies for shuffled data
        accuracies_shuffle = np.array([
            [np.trace(cm_shuffle[win_idx, rep_idx]) / np.sum(cm_shuffle[win_idx, rep_idx])
             for rep_idx in range(decoder_shuffle.n_repeats)]
            for win_idx in range(cm_shuffle.shape[0])
        ])  # Shape: (n_windows, n_permutations)

        # Mean accuracies across repetitions
        mean_accuracies_shuffle = np.mean(accuracies_shuffle, axis=1)  # Shape: (n_permutations, n_windows)

        # Store the accuracies in the dictionaries
        accuracies_true_per_roi[roi] = {
            'mean_accuracies_true': mean_accuracies_true,
            'accuracies_true': accuracies_true  # Optional: include individual accuracies
        }
        accuracies_shuffle_per_roi[roi] = {
            'mean_accuracies_shuffle': mean_accuracies_shuffle,
            'accuracies_shuffle': accuracies_shuffle  # Optional: include individual accuracies
        }

    return accuracies_true_per_roi, accuracies_shuffle_per_roi

make sure to use roi_labeled_arrays_no_nans if using the "subsample" balance method. Maybe these should be calculated together in one step...

In [49]:
for condition_comparison, strings_to_find in condition_comparisons.items():
    # Get accuracies for each ROI
    accuracies_true_per_roi, accuracies_shuffle_per_roi = get_accuracies_for_rois_time_window_decoding_jim(
        roi_labeled_arrays=roi_labeled_arrays,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        n_splits=2,
        n_repeats=2,           # Number of repeats for true decoding
        obs_axs=0,
        balance_method='subsample',
        random_state=42,
        window_size=None,  # Specify your window size in samples
        step_size=128,      # Specify your step size in samples
        n_permutations=2      # Number of permutations for shuffled decoding. Let's just do 10 for testing.
    )

Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 59, 641)
Condition ['c25.0', 'c75.0'] has 131 trials
Condition ['i25.0', 'i75.0'] has 72 trials


  0%|          | 0/10 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'